# Replikacija rezultata
### Fran Maznik 

#### Ucitavanje prethodno pripremljenih podataka

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('data/my_shot_logs.csv')
data.shape

(125751, 19)

In [3]:
data.head()

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,SHOT_DIST,PTS_TYPE,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,FGM,player_name,player_id
0,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,1,1,69,10.8,2,7.7,2,made,"Anderson, Alan",101187,1.3,1,brian roberts,203148
1,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,2,1,14,3.4,0,28.2,3,missed,"Bogdanovic, Bojan",202711,6.1,0,brian roberts,203148
2,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,3,1,0,0.0,3,10.1,2,missed,"Bogdanovic, Bojan",202711,0.9,0,brian roberts,203148
3,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,4,2,707,10.3,2,17.2,2,missed,"Brown, Markel",203900,3.4,0,brian roberts,203148
4,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,5,2,634,10.9,2,3.7,2,missed,"Young, Thaddeus",201152,1.1,0,brian roberts,203148


In [4]:
y = data["FGM"]  

In [5]:
data.drop(["FGM", "SHOT_RESULT"], axis=1, inplace=True)


Koristeni su algoritmi su: logistička regresija, svm, neuronska mreža, naivni bayesov klasifikator, random forests i boosting.
Za logističku regresiju, svm i neurosnku mrežu odbacujemo nebrojčane značajke


In [6]:
X = data.drop(
["GAME_ID", "MATCHUP", "LOCATION", "W", "CLOSEST_DEFENDER" ,"CLOSEST_DEFENDER_PLAYER_ID", "player_name", "player_id"]
, axis=1)


In [7]:
X.head()

,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,SHOT_DIST,PTS_TYPE,CLOSE_DEF_DIST
0,24,1,1,69,10.8,2,7.7,2,1.3
1,24,2,1,14,3.4,0,28.2,3,6.1
2,24,3,1,0,0.0,3,10.1,2,0.9
3,24,4,2,707,10.3,2,17.2,2,3.4
4,24,5,2,634,10.9,2,3.7,2,1.1


### Priprema podataka 
Podatci su podijeljeni u omjeru 95/5 za ucenje i ispitvanje

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.05)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


### 1. Logisticka regresija

In [9]:
from sklearn.metrics import  confusion_matrix

def my_confusion_matrix(y_true, y_predicted):
    labels = ["p", "n"]
    columns = [f"predicted {label}" for label in labels]
    index = [f"actual {label}" for label in labels]
    table = pd.DataFrame(confusion_matrix(y_true, y_predicted), columns=columns, index=index)
    return table


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression( ).fit(X_train,y_train)


In [11]:
accuracy_score(y_test, model.predict(X_test))

0.6154580152671756

Točnost u članku za logističku regresiju iznosi 59%

In [12]:
my_confusion_matrix(y_test, model.predict(X_test))

,predicted p,predicted n
actual p,2552,925
actual n,1493,1318


### 2. SVM

In [15]:
from sklearn.svm import LinearSVC
print(5)

5


In [22]:

model = LinearSVC( max_iter=2000).fit(X_train, y_train)

c:\Users\Korisnik\miniconda3\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [23]:
accuracy_score(y_test, model.predict(X_test))

0.6154580152671756

Točnost u članku za SVM iznosi 55%

In [24]:
my_confusion_matrix(y_test, model.predict(X_test))


,predicted p,predicted n
actual p,2551,926
actual n,1492,1319


### 3. Neuronske mreze

In [34]:
from sklearn.neural_network import MLPClassifier

In [40]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(50), max_iter = 3000)


In [41]:
clf.fit(X_train, y_train)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=50, max_iter=3000, solver='lbfgs')

In [42]:
accuracy_score(y_test, model.predict(X_test))


0.5742684478371501

### 4. Bayes

In [28]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler




X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.05)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [29]:
model = MultinomialNB()

model.fit(X_train, y_train)

MultinomialNB()

In [32]:
accuracy_score(y_test, model.predict(X_test))

0.5742684478371501

In [33]:
my_confusion_matrix(y_test, model.predict(X_test))


,predicted p,predicted n
actual p,3268,190
actual n,2487,343
